In [1]:
import pandas as pd
import datetime
import random
from collections import defaultdict

# Load data
startups = pd.read_csv("Startups.csv")
mentors = pd.read_csv("Mentor.csv")

# Fill empty fields with "any"
startups.fillna("any", inplace=True)
mentors.fillna("any", inplace=True)
print("[INFO] Data loaded and missing values filled.")

# Initialize time slots and schedules
start_time = datetime.datetime.strptime("11:00", "%H:%M")
end_time = datetime.datetime.strptime("14:00", "%H:%M")
time_slot_duration = datetime.timedelta(minutes=15)
gap_duration = datetime.timedelta(minutes=5)
print("[INFO] Time slots initialized.")

# Sector-wise distribution of startups
sector_startups = defaultdict(list)
for idx, row in startups.iterrows():
    sector_startups[row['sector']].append(idx)
print("[INFO] Sector-wise distribution of startups completed.")

# Mentor schedule and tracking
mentor_schedule = defaultdict(list)
mentor_startup_tracker = defaultdict(set)  # Tracks which startups each mentor has met
startup_mentor_count = defaultdict(int)    # Tracks how many times each startup has been allocated
allocated_startups = set()
print("[INFO] Mentor tracking and schedule initialized.")

# Probability distribution function
def get_startup_by_preference(mentor, preferences, bias=0.8):
    """
    Select a startup based on the mentor's preferences with a bias.
    """
    probabilities = [bias, (1 - bias) * 0.6, (1 - bias) * 0.4]  # Approximate 85% bias for first, split remaining
    for i, sector in enumerate(preferences):
        if sector in sector_startups and sector_startups[sector]:
            if random.random() <= probabilities[i]:  # Check probability
                return sector_startups[sector].pop(0)  # Allocate a startup
    return None  # No startup could be allocated from preferences

# Assign startups to mentors
def assign_startups():
    global allocated_startups
    print("[INFO] Starting the assignment of startups to mentors.")
    
    # Phase 1: Ensure all startups are allocated at least once
    while True:
        unallocated_startups = set(startups.index) - allocated_startups

        # Break if no unallocated startups are left
        if not unallocated_startups:
            print("[DEBUG] All startups have been allocated at least once.")
            break

        for _, mentor in mentors.iterrows():
            mentor_name = mentor['Name']
            preferences = [mentor['Sector 1'], mentor['Sector 2'], mentor['Sector 3']]

            # Allocate a startup based on preference probabilities
            current_time = start_time if not mentor_schedule[mentor_name] else datetime.datetime.strptime(
                mentor_schedule[mentor_name][-1][0], "%H:%M"
            ) + time_slot_duration + gap_duration
            current_time = max(current_time, start_time)  # Ensure time advances correctly

            if current_time >= end_time:
                print("[DEBUG] End time reached for mentor scheduling.")
                return

            startup_idx = get_startup_by_preference(mentor, preferences)
            if startup_idx is None:
                continue

            # Assign time slot and update tracking
            mentor_schedule[mentor_name].append((current_time.strftime("%H:%M"), startup_idx))
            mentor_startup_tracker[mentor_name].add(startup_idx)
            allocated_startups.add(startup_idx)
            startup_mentor_count[startup_idx] += 1
        
        print("[DEBUG] Completed one round of unallocated startups.")

    # Phase 2: Allow repetition with constraints
    while True:
        repeat_needed = False
        for _, mentor in mentors.iterrows():
            mentor_name = mentor['Name']
            preferences = [mentor['Sector 1'], mentor['Sector 2'], mentor['Sector 3']]

            current_time = start_time if not mentor_schedule[mentor_name] else datetime.datetime.strptime(
                mentor_schedule[mentor_name][-1][0], "%H:%M"
            ) + time_slot_duration + gap_duration

            if current_time >= end_time:
                print("[DEBUG] End time reached during repetition allocation.")
                return

            # Get startup ensuring it hasn't met the mentor more than once yet
            startup_idx = get_startup_by_preference(mentor, preferences)
            if startup_idx is None or startup_idx in mentor_startup_tracker[mentor_name]:
                continue  # Skip already met startups for this mentor

            # Limit the number of allocations to 5 per startup
            if startup_mentor_count[startup_idx] >= 5:
                continue

            # Assign time slot and update tracking
            mentor_schedule[mentor_name].append((current_time.strftime("%H:%M"), startup_idx))
            mentor_startup_tracker[mentor_name].add(startup_idx)
            startup_mentor_count[startup_idx] += 1
            repeat_needed = True
        if not repeat_needed:
            print("[DEBUG] Repetition allocation completed.")
            break

# Assign startups and display the schedule
assign_startups()

# Display results
print("\nMentor Schedules:\n")
for mentor, schedule in mentor_schedule.items():
    print(f"Mentor: {mentor}")
    for time, startup_idx in schedule:
        print(f"  Time: {time}, Startup Index: {startup_idx}, Name: {startups.loc[startup_idx, 'Name']}")
    print("\n")

# Display the count of mentor allocations for each startup
print("\nStartup Allocation Counts:\n")
for startup_idx, count in startup_mentor_count.items():
    print(f"Startup Index: {startup_idx}, Name: {startups.loc[startup_idx, 'Name']}, Allocations: {count}")

print("[INFO] Mentor scheduling completed.")


[INFO] Data loaded and missing values filled.
[INFO] Time slots initialized.
[INFO] Sector-wise distribution of startups completed.
[INFO] Mentor tracking and schedule initialized.
[INFO] Starting the assignment of startups to mentors.
[DEBUG] Completed one round of unallocated startups.
[DEBUG] Completed one round of unallocated startups.
[DEBUG] Completed one round of unallocated startups.
[DEBUG] Completed one round of unallocated startups.
[DEBUG] Completed one round of unallocated startups.
[DEBUG] Completed one round of unallocated startups.
[DEBUG] Completed one round of unallocated startups.
[DEBUG] Completed one round of unallocated startups.
[DEBUG] Completed one round of unallocated startups.
[DEBUG] Completed one round of unallocated startups.
[DEBUG] All startups have been allocated at least once.
[DEBUG] Repetition allocation completed.

Mentor Schedules:

Mentor: Mentor 1
  Time: 11:00, Startup Index: 40, Name: Startup 41
  Time: 11:20, Startup Index: 69, Name: Startup 7